# RAG For Mantine Documentation

In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core.schema import Document as LlamaDocument, MetadataMode
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from pathlib import Path

In [2]:
from rag_service.db import DatabaseManager
from rag_service.models import Document, Chunk

In [3]:
local_session = DatabaseManager.get_session_factory()

In [4]:
Path.cwd()
doc_path = Path("../documents/mantine-llms-full.txt")
doc_path.exists()

True

## Ingestion

In [5]:
# Load Mantine documentation

reader = SimpleDirectoryReader(input_files=[doc_path])
documents = reader.load_data()
mantine_documentation = documents[0]

print(len(mantine_documentation.text))
print(mantine_documentation.metadata['creation_date'])



2252482
2026-01-12


In [6]:
from google.genai.types import EmbedContentConfig

In [7]:
from dotenv import load_dotenv
import os
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
embedding_model = GoogleGenAIEmbedding(
    model_name="gemini-embedding-001", 
    api_key=GEMINI_API_KEY,
    embedding_config=EmbedContentConfig(output_dimensionality=1536)
    )

In [8]:
doc_row = Document(
    source='https://mantine.dev/llms-full.txt',
    title='Mantine UI Library - Complete Documentation',
    embedding_model='gemini-embedding-001'
)

In [ ]:
async with local_session() as session:
    async with session.begin():
        session.add(doc_row)

In [11]:
import uuid
doc_id = "567375ed-9bad-40d1-bb2e-7c93bb041f35"
doc_uuid = uuid.UUID(doc_id)

In [12]:
pipeline = IngestionPipeline(
    transformations=[
        MarkdownNodeParser(),
        embedding_model   
    ]
)

nodes = pipeline.run(documents=[mantine_documentation], show_progress=True)

/home/ubuntu/projects/RAG_service/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 4068/4068 [02:37<00:00, 25.88it/s]


In [15]:
from sqlalchemy.dialects.postgresql import insert

async with local_session() as session:
    async with session.begin():
        rows = [
            {
                "chunk_index": i,
                "document_id": doc_uuid,
                "content": node.get_content(metadata_mode=MetadataMode.NONE),
                "embedding": node.get_embedding(),
                "chunk_metadata": node.metadata,
            }
            for i, node in enumerate(nodes)
        ]

        stmt = insert(Chunk).values(rows)
        stmt = stmt.on_conflict_do_update(
            index_elements=["document_id", "chunk_index"],
            set_={
                "content": stmt.excluded.content,
                "embedding": stmt.excluded.embedding,
                "chunk_metadata": stmt.excluded.chunk_metadata,
            },
        )
        await session.execute(stmt)


## Retrieval

In [16]:
query = "How do you create custom button variants in Mantine?"

query_embedding = embedding_model.get_query_embedding(query)

In [28]:
from sqlmodel import select


async with local_session() as session:

    dist = Chunk.embedding.cosine_distance(query_embedding).label("dist")

    stmt = (
        select(
            Chunk.id,
            Chunk.chunk_index,
            Chunk.content,
            Chunk.chunk_metadata,
            Document.title,
            Document.source,
            dist,
        )
        .join(Document, Chunk.document_id == Document.id)
        .order_by(dist)
        .limit(10)
    )

    result = await session.execute(stmt)
    rows = result.all()  # tuples of the selected columns


    
        